In [ ]:
#in this part we generate 500 matrices for testing

import numpy as np
import networkx as nx
import random

prob_matrix = list() #probability that path is free
lens_matrix = list() #lengths of roads between edges
while len(prob_matrix) < 500:
  n = 50
  m = 50
  a = np.random.randint(50, size=(m,n))
  a = (a + a.T)
  b = np.random.randint(50, size=(m,n))
  b = (b + b.T)
  Ga = nx.Graph(a)
  Gb = nx.Graph(b)
  for i in range(m):
    for j in range(n):
        if random.random() < 0.5:
          a[i][j] = 0
          b[i][j] = 1000 #represents infinity
          if Gb.has_edge(min(i, j), max(i, j)):
            Gb.remove_edge(min(i, j), max(i, j))
  for i in range(m):
    a[i][i] = 0
    b[i][i] = 1000
    if Gb.has_edge(i,i):
      Gb.remove_edge(i,i)
  if nx.has_path(Gb, 0, 49):
    lens_matrix.append(b)
    prob_matrix.append(a)

print(lens_matrix[6])
print(prob_matrix[6])

[[1000 1000 1000 ... 1000   73 1000]
 [  34 1000 1000 ...   42   51   46]
 [1000   58 1000 ... 1000   37 1000]
 ...
 [  57 1000 1000 ... 1000   28 1000]
 [1000   51   37 ...   28 1000   15]
 [  22   46   18 ...   45 1000 1000]]
[[ 0  0  0 ...  0 24  0]
 [75  0  0 ... 52 60 11]
 [ 0 53  0 ...  0 39  0]
 ...
 [22  0  0 ...  0 97  0]
 [ 0 60 39 ... 97  0 50]
 [48 11 83 ... 40  0  0]]


In [ ]:
import time


#function, that generates real maps
def map_based_on_prob(map, prob):
  set_of_maps_for_testing = list()
  while len(set_of_maps_for_testing) < 10:
    n = len(map)
    m = len(map[0])
    a = np.random.randint(50, size=(m,n))
    a = (a + a.T)
    G = nx.Graph(a)
    for i in range(m):
      for j in range(n):
        if random.random() > prob[i][j]/100:
          a[i][j] = 1000
          if G.has_edge(min(i, j), max(i, j)):
            G.remove_edge(min(i, j), max(i, j))
    if nx.has_path(G, 0, 49):
      set_of_maps_for_testing.append(a)
  return set_of_maps_for_testing

#function, that finds most persistent path
def most_persistent_path(map, s, t):
  for e in range(100, 0, -1):
    G = nx.Graph()
    for i in range(len(map)):
      for j in range(i, len(map)):
        if (map[i][j] != 0) and (map[i][j] >= e):
          G.add_node(i)
          G.add_node(j)
          G.add_edge(i, j, length = map[i][j])
        elif (map[i][j] == 0):
          if G.has_edge(i, j):
            G.remove_edge(i, j)
          if G.has_edge(j, i):
            G.remove_edge(j, i)
    if (s in G.nodes()) and (t in G.nodes()) and nx.has_path(G, s, t):
      return nx.shortest_path(G, s, t)
      break
    if e == 0:
      return False
  return False

def my_strategy(given_map, prob_matrix, real_map):
  coordinate = 0
  final_path = list()
  while coordinate != 49:
    path = most_persistent_path(prob_matrix, coordinate, 49)
    if path == False:
      return False
    #print('we are trying path', path, 'the road is free with probability', prob_matrix[path[0]][path[1]])
    final_path.append([path[0], path[1]])
    #print(final_path, coordinate)
    if real_map[path[0]][path[1]] != 1000:
      coordinate = path[1]
      prob_matrix[path[0]][path[1]] = 100
      prob_matrix[path[1]][path[0]] = 100
      #print(coordinate)
    else:
      prob_matrix[path[0]][path[1]] = 0
      prob_matrix[path[1]][path[0]] = 0
      final_path.append([path[1], path[0]])
    if len(final_path) > 1000:
      print('Oops... something went wrong...', final_path, 'path we are trying', path)
  return final_path


all_times = list()
all_final_paths = list()
all_short_paths = list()
for i in range(len(lens_matrix)):
  test_set = map_based_on_prob(lens_matrix[i], prob_matrix[i])
  for j in range(len(test_set)):
    real_map = test_set[j]
    final_path = my_strategy(lens_matrix[i], prob_matrix[i], real_map)
    if final_path != False:
      G = nx.Graph(real_map)
      start = time.time()
      for k in range(50):
        for l in range(50):
          if (real_map[k][l] == 1000):
            if G.has_edge(k, l):
              G.remove_edge(k, l)
            if G.has_edge(l, k):
              G.remove_edge(l, k)
      all_short_paths.append(nx.shortest_path(G, 0, 49))
      all_final_paths.append(final_path)
      end = time.time() - start
      all_times.append(end)




final path for try number 0 and some test map number 0 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 1 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 2 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 3 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 4 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 5 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 6 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 7 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 8 is [[0, 34], [34, 49]]
final path for try number 0 and some test map number 9 is [[0, 34], [34, 49]]
final path for try number 1 and some test map number 0 is [[0, 49]]
final path for try number 1 and some test map number 1 is [[0, 49]]
final path for try number 1 and some test map number 2 is [[0, 49]]
final path for t

In [ ]:
all_lens = list()
short_lens = list()

for j in range(len(all_final_paths)):
  path = all_final_paths[j]
  l = 0
  map = lens_matrix[j//10]
  for edge in path:
    l += map[edge[0]][edge[1]]
  all_lens.append(l)

for j in range(len(all_short_paths)):
  path = all_short_paths[j]
  l = 0
  map = lens_matrix[j//10]
  for k in range(len(path)-1):
    l += map[path[k]][path[k+1]]
  short_lens.append(l)

In [ ]:
all_lens_2 = list()
short_lens_2 = list()

for path in all_final_paths:
  all_lens_2.append(len(path))

for path in all_short_paths:
  short_lens_2.append(len(path))

comp_ratio_2 = list()
for i in range(len(all_lens)):
  comp_ratio_2.append(all_lens_2[i]/short_lens_2[i])

In [ ]:
comp_ratio = list()
for i in range(len(all_lens)):
  comp_ratio.append(all_lens[i]/short_lens[i])